# 实验 3 - BP 算法

## 基本信息

* 课程：深度学习引论
* 老师：GUO QUAN
* 学生：王昊霖
* ID：2020141440041


In [605]:
import numpy as np
import random
import copy

In [606]:
# prepare trainning data set
data_set = np.array([[1,0],[0,1],[1,1],[0,0]])
labels = np.array([1,1,0,0])

In [607]:
# initial parameters and weights
alpha = 0.01
epochs = 100000
batch_size =2
w1 =np.random.randn(3,2)
w2 =np.random.randn(1,3)
W = np.array([w1,w2])
J = np.zeros((1,4))

<ipython-input-607-fac142b9f1ba>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  W = np.array([w1,w2])


构造数据迭代器，返回 `batch_size` 个样本

In [608]:
def data_iter(data_set,labels,batch_size):
    indices =list(range(len(data_set)))
    random.shuffle(indices)
    for i in range(0,len(indices),batch_size):
        batch_indices =indices[i:i+batch_size]
        yield data_set[batch_indices],labels[batch_indices]

print(next(data_iter(data_set,labels,batch_size)))

(array([[0, 0],
       [1, 1]]), array([0, 0]))


In [609]:
# retain the forward result
a_para = []
z_para = []
       

In [610]:
def sigmoid(inx):
   if inx.any() >= 0: 
        return 1.0 / (1 + np.exp(-inx))
   else:
        return np.exp(inx) / (1 + np.exp(inx))

def purline(x):
    return x

def derivative_of_sigmoid(x):
    return sigmoid(x)*(1- sigmoid(x))


def loss_function(x,y):
    return 1./2*(x-y)**2

def forward(w,a):
    f_list = [sigmoid,purline]
    for w_i,f in zip(w,f_list):
        z = w_i@a
        a = f(z)
        z_para.append(copy.deepcopy(z))
        a_para.append(copy.deepcopy(a))
        
    return a

def backward(w,delta):
    for index,(w_i,z) in enumerate(zip(w[-1:],z_para[-2:])):
        F = np.diag(derivative_of_sigmoid(z))
        delta = F @ w_i.T@delta
    return delta


## 在线BP算法

In [611]:
def online_train():
  global W,w1,w2
  for t in range(epochs):
      for index, (data,label) in enumerate(zip(data_set,labels)):
        a_para.append(data.T)
        a =  forward(W,data.T)
        loss = loss_function(a,label)
    
        delta_L = (a - label)*(z_para[-1])
        delta = backward(W,delta_L)
      
        dw2 = np.dot(delta_L.reshape((1,1)),np.array(a_para[1]).reshape((1,3)))
        dw1 =np.dot(delta.reshape((3,1)),np.array(a_para[0]).reshape((1,2)))

        w2 = w2 -alpha*dw2
        w1 = w1- alpha*dw1
      

        W = np.array([w1,w2])

        a_para.clear()
      if t%1000 == 0:
          print(f"epoch: {t},loss :{loss[0]:.3f}")
# online_train()
    

## 批处理反向传播算法

In [612]:
for t in range(epochs):
    J = [np.zeros_like(w1),np.zeros_like(w2)]
    for minbatch in data_iter(data_set,labels,batch_size):
      for data,label in zip(minbatch[0],minbatch[1]):
        a_para.append(data.T)
        a =  forward(W,data.T)
        loss = loss_function(a,label)
    
        delta_L = (a - label)*(z_para[-1])
        delta = backward(W,delta_L)
      
        dw2 = np.dot(delta_L.reshape((1,1)),np.array(a_para[1]).reshape((1,3)))
        dw1 =np.dot(delta.reshape((3,1)),np.array(a_para[0]).reshape((1,2)))

        J[1]+=dw2
        J[0]+=dw1
      w2 = w2 -alpha*J[1]/batch_size
      w1 = w1- alpha*J[0]/batch_size
    

    W = np.array([w1,w2])

    a_para.clear()
    if t%1000 == 0:
        print(f"epoch: {t},loss :{loss[0]:.3f}")
    

<ipython-input-612-992ec929c1e0>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  W = np.array([w1,w2])


epoch: 0,loss :0.041
epoch: 1000,loss :0.193
epoch: 2000,loss :0.013
epoch: 3000,loss :0.261
epoch: 4000,loss :0.006
epoch: 5000,loss :0.005
epoch: 6000,loss :0.224
epoch: 7000,loss :0.051
epoch: 8000,loss :0.117
epoch: 9000,loss :0.045
epoch: 10000,loss :0.042
epoch: 11000,loss :0.207
epoch: 12000,loss :0.196
epoch: 13000,loss :0.070
epoch: 14000,loss :0.069
epoch: 15000,loss :0.038
epoch: 16000,loss :0.179
epoch: 17000,loss :0.194
epoch: 18000,loss :0.185
epoch: 19000,loss :0.038
epoch: 20000,loss :0.045
epoch: 21000,loss :0.051
epoch: 22000,loss :0.035
epoch: 23000,loss :0.042
epoch: 24000,loss :0.043
epoch: 25000,loss :0.044
epoch: 26000,loss :0.043
epoch: 27000,loss :0.039
epoch: 28000,loss :0.036
epoch: 29000,loss :0.040
epoch: 30000,loss :0.047
epoch: 31000,loss :0.043
epoch: 32000,loss :0.153
epoch: 33000,loss :0.046
epoch: 34000,loss :0.029
epoch: 35000,loss :0.047
epoch: 36000,loss :0.043
epoch: 37000,loss :0.046
epoch: 38000,loss :0.027
epoch: 39000,loss :0.152
epoch: 40000,

## 测试结果

In [613]:
for index, (data,label) in enumerate(zip(data_set,labels)):
    a =  forward(W,data.T)
    print(f"simple ({data},{label}) is classified  {round(a[0])}")

simple ([1 0],1) is classified  1
simple ([0 1],1) is classified  1
simple ([1 1],0) is classified  0
simple ([0 0],0) is classified  0
